In [1]:
import db_access as dba
import flylib
reload(dba)
fly_db = dba.get_db()

In [2]:
#ephys_group = [111,112,114,115,116,117,118,122,123,124,125,130]
ephys_group = [111,112,114,123,125]
#ephys_group = 
swarm = flylib.Squadron(fly_db,ephys_group)

In [4]:
%pylab

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [6]:
lrb.get_sequences()

{1: <flylib.HSVSequence at 0x7f5da4bcd610>,
 2: <flylib.HSVSequence at 0x7f5da4bcd7d0>,
 3: <flylib.HSVSequence at 0x7f5da4bcd8d0>,
 4: <flylib.HSVSequence at 0x7f5da4bcd9d0>,
 5: <flylib.HSVSequence at 0x7f5da4bcdad0>,
 6: <flylib.HSVSequence at 0x7f5da4bcdbd0>}

In [11]:
import cPickle
for fly in swarm.flies:
    figure()
    print fly.fly_num
    lrb = fly.experiments['lr_blob_expansion']
    ### get the spike time data
    f = open(fly.fly_path + 'spike_mask.cpkl');spike_mask = cPickle.load(f);f.close()
    f = open(fly.fly_path + 'spike_pool.cpkl');spike_pool = cPickle.load(f);f.close()
    for seq_key in lrb.get_sequences().keys():
        seq =  lrb.get_sequences()[seq_key]
        b1_spikes = spike_pool[spike_mask]
        phases = get_seq_phases(seq,b1_spikes)
        dev = mean(np.array(seq.seq_record['wb_mtrx']['theta_R'])[:,500:525],axis = 1)
        amp = mean(np.array(seq.seq_record['wb_mtrx']['phi_R'])[:,500:525],axis = 1)
        idx = ~isnan(phases)
        scatter(rad2deg(amp[idx]),rad2deg(dev[idx]),c = cm.cool(phases[idx]/(2*pi)))
        gca().set_ybound(-50,10)
        gca().set_xbound(10,-50)

111
112
114
123
125


In [10]:
#idx = ~isnan(np.array(seq.seq_record['wb_mtrx']['axon_sample_times'])[0])
def get_seq_phases(seq,b1_spikes):
    spike_idxs = []
    for wb in seq.seq_record['wb_mtrx']['axon_sample_times']:
        wb = array(wb)
        wb_times = wb[250:750]
        wb_times = wb_times[~isnan(wb_times)]
        try:
            spike_time = b1_spikes[argwhere((b1_spikes > wb_times[0]) & (b1_spikes < wb_times[-1]))[0][0]]
            spike_idx = argwhere(spike_time < wb_times)[0]
            spike_idxs.append(spike_idx)
        except IndexError:
            spike_idxs.append(np.nan)
    return np.squeeze(np.array(spike_idxs)*(2*pi/500)).astype(float)

In [13]:
plot(lrb.exp_record['axon_data']['times'],lrb.exp_record['axon_data']['AMsysCh1'])
plot(b1_spikes[:1000],ones_like(b1_spikes[:1000]),'o')

In [16]:
seqs = lrb.get_sequences()

In [17]:
seqs.keys()

[7, 8, 9, 10, 11, 12]

In [15]:
fly = swarm.flies[1]
lrb = fly.experiments['lr_blob_expansion']
seq_key = lrb.sequences.keys()[0]
seq = lrb.sequences[seq_key+0]
dev_mtrx = np.array(seq.seq_record['wb_mtrx']['theta_R'])
str_mtrx = np.array(seq.seq_record['wb_mtrx']['phi_R'])
rot_mtrx = np.array(seq.seq_record['wb_mtrx']['eta_R'])
print seq.seq_record

AttributeError: 'HSVExperiment' object has no attribute 'sequences'

In [176]:
phases = np.linspace(0,2*np.pi,500)
p_dev_tether = flylib.fit_fourier(dev_mtrx[:20,500:1000],p_dev)
p_str_tether = flylib.fit_fourier(str_mtrx[:20,500:1000],p_str)
p_rot_tether = flylib.fit_fourier(rot_mtrx[:20,500:1000],p_rot)

steadywb_dev = flylib.fourier(phases,p_dev_tether)
steadywb_str = flylib.fourier(phases,p_str_tether)
steadywb_rot = flylib.fourier(phases,p_rot_tether)

diff_dev = (steadywb_dev[:,newaxis] - rad2deg(dev_mtrx[:,500:1000]).T).T
diff_str = (steadywb_str[:,newaxis] - rad2deg(str_mtrx[:,500:1000]).T).T
diff_rot = (steadywb_rot[:,newaxis] - rad2deg(rot_mtrx[:,500:1000]).T).T
subplot(3,1,1)
plot(mean(diff_dev,axis= 1))
subplot(3,1,2)
plot(mean(diff_str,axis= 1))
subplot(3,1,3)
plot(mean(diff_rot,axis= 1))

In [202]:
stro_R = 'phi_R'
stro_L = 'phi_L'
dev_R = 'theta_R'
dev_L = 'theta_L'
rot_R = 'eta_R'
rot_L = 'eta_L'

sorted_amp = dict()
sorted_dev = dict()
sorted_rot = dict()
sorted_phys = dict()

ephys_group = [115,114,112]
swarm = flylib.Squadron(fly_db,ephys_group)
    
for kin_dict,keys in zip([sorted_amp,sorted_dev,sorted_rot],
                   [[stro_L, stro_R],
                    [dev_L,  dev_R],
                    [rot_L,   rot_R]]):
    kin_dict['L_wing_R_exp'] = list()
    kin_dict['R_wing_R_exp'] = list()
    kin_dict['L_wing_L_exp'] = list()
    kin_dict['R_wing_L_exp'] = list()

    for fly in swarm.flies:#,111]:# fly_db.keys():
        lrb = fly.experiments['lr_blob_expansion']
        seqs = [lrb.sequences[k] for k in lrb.sequences.keys()]
        #seqs = fly_db[key]['experiments']['lr_blob_expansion']['wbkin_sequences']
        for seq in seqs:
            if seq.seq_record['expan_pol'].value == 90.0:
                kin_dict['L_wing_R_exp'].append(np.array(seq.seq_record['wb_mtrx'][keys[0]]))
                kin_dict['R_wing_R_exp'].append(np.array(seq.seq_record['wb_mtrx'][keys[1]]))
            if seq.seq_record['expan_pol'].value == 270.0:
                kin_dict['L_wing_L_exp'].append(np.array(seq.seq_record['wb_mtrx'][keys[0]]))
                kin_dict['R_wing_L_exp'].append(np.array(seq.seq_record['wb_mtrx'][keys[1]]))

sorted_phys['L_wing_R_exp'] = list()
sorted_phys['L_wing_L_exp'] = list()
for fly in swarm.flies:#fly_db.keys(): 
    lrb = fly.experiments['lr_blob_expansion']
    seqs = [lrb.sequences[k] for k in lrb.sequences.keys()]
    #seqs = fly_db[key]['experiments']['lr_blob_expansion']['wbkin_sequences']
    for seq in seqs:
        #print seq.seq_record['expan_pol'].value == 90.0
        if seq.seq_record['expan_pol'].value == 90.0:
            sorted_phys['L_wing_R_exp'].append(np.array(seq.seq_record['wb_mtrx']['AMsysCh1']))
        if seq.seq_record['expan_pol'].value == 270.0:
            sorted_phys['L_wing_L_exp'].append(np.array(seq.seq_record['wb_mtrx']['AMsysCh1']))

In [248]:
#Plot the correlation between kine params and spike phase
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
phases = np.linspace(0,2*np.pi,500)

str_mtrx = vstack(sorted_amp['L_wing_L_exp'])
dev_mtrx = vstack(sorted_dev['L_wing_L_exp'])
rot_mtrx = vstack(sorted_rot['L_wing_L_exp'])

p_dev_tether = flylib.fit_fourier(dev_mtrx[:20,500:1000],p_dev)
p_str_tether = flylib.fit_fourier(str_mtrx[:20,500:1000],p_str)
p_rot_tether = flylib.fit_fourier(rot_mtrx[:20,500:1000],p_rot)

steadywb_dev = flylib.fourier(phases,p_dev_tether)
steadywb_amp = flylib.fourier(phases,p_str_tether)
steadywb_rot = flylib.fourier(phases,p_rot_tether)

amp_residual_sum = None
dev_residual_sum = None
rot_residual_sum = None
spike_phases = None

for amp_sequence,dev_sequence,rot_sequence,phys_sequence in zip(sorted_amp['L_wing_L_exp'],
                                                   sorted_dev['L_wing_L_exp'],
                                                   sorted_rot['L_wing_L_exp'],
                                                   sorted_phys['L_wing_L_exp']):
    
    #amp_sequence = amp_sequence[:,500:1000]
    #dev_sequence = dev_sequence[:,500:1000]
    #rot_sequence = rot_sequence[:,500:1000]
    
    amp_sequence = amp_sequence[:,500:1000]
    dev_sequence = dev_sequence[:,500:1000]
    rot_sequence = rot_sequence[:,500:1000]
    
    phys_sequence = phys_sequence[:,500:1000]

    amp_residuals = rad2deg(amp_sequence)-steadywb_amp[0:500]
    dev_residuals = rad2deg(dev_sequence)-steadywb_dev[0:500]
    rot_residuals = rad2deg(rot_sequence)-steadywb_rot[0:500]
    
    if spike_phases is None:
        spike_phases = phases[argmax(phys_sequence,axis = 1)]
    else:
        spike_phases = hstack((spike_phases,phases[argmax(phys_sequence,axis = 1)]))
    
    if amp_residual_sum is None:
        amp_residual_sum = nanmean(amp_residuals, axis = 1)
    else:
        amp_residual_sum = hstack((amp_residual_sum, nanmean(amp_residuals, axis = 1)))
    
    if dev_residual_sum is None:
        dev_residual_sum = nanmean(dev_residuals, axis = 1)
    else:
        dev_residual_sum = hstack((dev_residual_sum, nanmean(dev_residuals, axis = 1)))
    
    if rot_residual_sum is None:
        rot_residual_sum = nanmean(rot_residuals, axis = 1)
    else:
        rot_residual_sum = hstack((rot_residual_sum, nanmean(rot_residuals, axis = 1)))
#amp_residual_sum /= std(amp_residual_sum)
#dev_residual_sum /= std(dev_residual_sum)
#rot_residual_sum /= std(rot_residual_sum)
ax0 = subplot(3,1,1)
cmap = cm.spectral
scatter(amp_residual_sum,dev_residual_sum,c = spike_phases,norm=Normalize(0,2*pi),cmap = cmap)
#ax0.axis('equal')
ax1 = subplot(3,1,2,sharex = ax0,sharey = ax0)
scatter(amp_residual_sum,rot_residual_sum,c = spike_phases,norm=Normalize(0,2*pi),cmap = cmap)
#ax1.axis('equal')
ax2 = subplot(3,1,3,sharex = ax0,sharey = ax0)
scatter(dev_residual_sum,rot_residual_sum,c = spike_phases,norm=Normalize(0,2*pi),cmap = cmap)
#ax2.axis('equal')
cax = gcf().add_axes([0.9, 0.1, 0.03, 0.8])
colorbar(cax=cax)


<matplotlib.colorbar.Colorbar instance at 0x139c9ddd0>

In [262]:
imshow(phases[:,newaxis],cmap = cmap,aspect ='auto')
ax = gca()

In [263]:
ax.set_yticks(linspace(0,500,5))
#ax.set_yticklabels(linspace(0,2*pi,5))
draw()

In [91]:
figure(figsize = (8,10))

######################
######################
#######################
fly = swarm.flies[1]
lrb = fly.experiments['lr_blob_expansion']
skey = lrb.sequences.keys()[1]
cam_times = np.array(lrb.sequences[skey].seq_record['wbkin']['photron_sample_times'])
phi_R = rad2deg(np.array(lrb.sequences[skey].seq_record['wbkin']['phi_R']))
phi_L = rad2deg(np.array(lrb.sequences[skey].seq_record['wbkin']['phi_L']))
eta_R = rad2deg(np.array(lrb.sequences[skey].seq_record['wbkin']['eta_R'])-pi/2)
eta_L = rad2deg(np.array(lrb.sequences[skey].seq_record['wbkin']['eta_L'])-pi/2)
theta_R = rad2deg(np.array(lrb.sequences[skey].seq_record['wbkin']['theta_R']))
theta_L = rad2deg(np.array(lrb.sequences[skey].seq_record['wbkin']['theta_L']))

import sys
import os
import cPickle
f = open(fly.fly_path + 'spike_mask.cpkl')
spike_mask = cPickle.load(f)
f.close()
f = open(fly.fly_path + 'spike_pool.cpkl')
spike_pool = cPickle.load(f)
f.close()

seq = lrb.sequences[skey]
axon_epoch = np.array(seq.seq_record['wbkin']['axon_epoch']).astype(int)
ephys = np.array(lrb.exp_record['axon_data']['AMsysCh1'])[axon_epoch]
stim = np.array(lrb.exp_record['axon_data']['Xpos'])[axon_epoch]
phys_times = np.array(lrb.exp_record['axon_data']['times'])[axon_epoch]
b1_spikes = spike_pool[spike_mask]
b1_spikes = b1_spikes[(b1_spikes>phys_times[0]) & (b1_spikes<phys_times[-1])]

spike_frames = digitize(b1_spikes,cam_times)

ax0 = subplot(10,1,1)
plot(phys_times-phys_times[0],stim,color = 'k')
     
subplot(10,1,2,sharex = ax0)
plot(phys_times-phys_times[0],ephys,color = 'b')

subplot(10,1,3,sharex = ax0)
plot(cam_times-cam_times[0],phi_R,'r')
plot(cam_times-cam_times[0],phi_L,'b')
plot(cam_times[spike_frames]-cam_times[0],phi_L[spike_frames],'o',color = 'b')
gca().set_ybound(-120,120)

subplot(10,1,4,sharex = ax0)
plot(cam_times-cam_times[0],eta_L,'r')
plot(cam_times-cam_times[0],eta_R,'b')
gca().set_ybound(-120,120)

subplot(10,1,5,sharex = ax0)
plot(cam_times-cam_times[0],theta_R,'r')
plot(cam_times-cam_times[0],theta_L,'b')
gca().set_ybound(-60,60)
gca().set_xbound(0.05,0.225)

fly = swarm.flies[11]
lrb = fly.experiments['lr_blob_expansion']
skey = lrb.sequences.keys()[0]
cam_times = np.array(lrb.sequences[skey].seq_record['wbkin']['photron_sample_times'])
phi_R = rad2deg(np.array(lrb.sequences[skey].seq_record['wbkin']['phi_R']))
phi_L = rad2deg(np.array(lrb.sequences[skey].seq_record['wbkin']['phi_L']))
eta_R = rad2deg(np.array(lrb.sequences[skey].seq_record['wbkin']['eta_R'])-pi/2)
eta_L = rad2deg(np.array(lrb.sequences[skey].seq_record['wbkin']['eta_L'])-pi/2)
theta_R = rad2deg(np.array(lrb.sequences[skey].seq_record['wbkin']['theta_R']))
theta_L = rad2deg(np.array(lrb.sequences[skey].seq_record['wbkin']['theta_L']))

##################
##################
##################

f = open(fly.fly_path + 'spike_mask.cpkl')
spike_mask = cPickle.load(f)
f.close()
f = open(fly.fly_path + 'spike_pool.cpkl')
spike_pool = cPickle.load(f)
f.close()

seq = lrb.sequences[skey]
axon_epoch = np.array(seq.seq_record['wbkin']['axon_epoch']).astype(int)
ephys = np.array(lrb.exp_record['axon_data']['AMsysCh1'])[axon_epoch]
stim = np.array(lrb.exp_record['axon_data']['Xpos'])[axon_epoch]
phys_times = np.array(lrb.exp_record['axon_data']['times'])[axon_epoch]
b1_spikes = spike_pool[spike_mask]
b1_spikes = b1_spikes[(b1_spikes>phys_times[0]) & (b1_spikes<phys_times[-1])]

spike_frames = digitize(b1_spikes,cam_times)

subplot(10,1,6)
plot(phys_times-phys_times[0],stim,color = 'k')
     
subplot(10,1,7,sharex = ax0)
plot(phys_times-phys_times[0],ephys*-1,color = 'b')

subplot(10,1,8,sharex = ax0)
plot(cam_times-cam_times[0],phi_R,'r')
plot(cam_times-cam_times[0],phi_L,'b')
plot(cam_times[spike_frames]-cam_times[0],phi_L[spike_frames],'o',color = 'b')
gca().set_ybound(-120,120)

subplot(10,1,9,sharex = ax0)
plot(cam_times-cam_times[0],eta_L,'r')
plot(cam_times-cam_times[0],eta_R,'b')
gca().set_ybound(-120,120)

subplot(10,1,10,sharex = ax0)
plot(cam_times-cam_times[0],theta_R,'r')
plot(cam_times-cam_times[0],theta_L,'b')
gca().set_ybound(-60,60)
gca().set_xbound(0.05,0.225)


IndexError: list index out of range

In [6]:
for fly in [swarm.flies[1]]:
    lrb = fly.experiments['lr_blob_expansion']
    #lrb.import_sequence_data()
    lrb.sync_sequences()
    fly_db.flush()

RuntimeError: unable to create link (Links: Unable to initialize object)

In [39]:
fly_db.close()

<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>


In [37]:
cdict = {'red':  ((0.0,0.0,0.0),(1.0,0.1,0.1)),
         'blue': ((0.0,0.0,0.0),(1.0,1.0,1.0)),
         'green':((0.0,0.0,0.0),(1.0,0.1,0.1)),}
my_cmap = matplotlib.colors.LinearSegmentedColormap('my_colormap',cdict,256)

for snum in [0,4]:
    figure(figsize = (8,10))
    fly = swarm.flies[1]
    lrb = fly.experiments['lr_blob_expansion']
    skey = lrb.sequences.keys()[snum]
    seq = lrb.sequences[skey]
    eta_L = np.rad2deg(np.array(seq.seq_record['wb_mtrx']['eta_L']))
    theta_L = np.rad2deg(np.array(seq.seq_record['wb_mtrx']['theta_L']))
    phi_L = np.rad2deg(np.array(seq.seq_record['wb_mtrx']['phi_L']))
    ephys = np.array(seq.seq_record['wb_mtrx']['AMsysCh1'])
    numstrks = shape(eta_L)[0]
    cbar = arange(numstrks)[:,newaxis]
    pcolor(cbar,cmap = my_cmap,edgecolors='w')
    alpha = 1.0
    phases = np.linspace(-2*pi,4*pi,1500)
    for strk in range(numstrks)[::2]:
        c = my_cmap((strk/(float(numstrks))))
        ax0 = subplot(4,1,1)
        plot(phases,phi_L[strk,:],color = c,alpha = alpha)
        gca().set_ybound(-100,100)
        gca().set_yticks(linspace(-100,100,5))
        subplot(4,1,2,sharex = ax0)
        plot(phases,theta_L[strk,:],color = c,alpha = alpha)
        gca().set_ybound(-100,100)
        gca().set_yticks(linspace(-40,40,5))
        subplot(4,1,3,sharex = ax0)
        plot(phases,eta_L[strk,:],color = c,alpha = alpha)
        gca().set_ybound(-100,100)
        gca().set_yticks(linspace(-100,100,5))
        subplot(4,1,4,sharex = ax0)
        plot(phases,ephys[strk,:],color = c,alpha = alpha)
        gca().set_ybound(-100,100)
        gca().set_yticks(linspace(-100,100,5))
    gca().set_xticks([-2*pi,-1*pi,0,pi,2*pi,3*pi,4*pi])
    gca().set_xbound([-2*pi,4*pi])

In [34]:
seq = lrb.sequences[12]

In [35]:
seq.calc_kine_phases()

In [36]:
seq.calc_wb_mtrx()

In [63]:
cdict = {'blue':  ((0.0,0.1,0.0),(1.0,0.0,0.0)),
         'red':   ((0.0,1.0,1.0),(1.0,0.0,0.0)),
         'green': ((0.0,0.1,0.0),(1.0,0.0,0.0)),}
#cdict = {'red':  ((1.0,0.1,0.1),(0.0,0.0,0.0)),
#         'blue': ((1.0,1.0,1.0),(0.0,0.0,0.0)),
#         'green':((1.0,0.1,0.1),(0.0,0.0,0.0)),}

#pcolor(cbar,cmap = my_cmap,edgecolors='w')
alpha = 0.5
for snum in [1,4]:
    figure(figsize = (8,10))
    fly = swarm.flies[1]
    lrb = fly.experiments['lr_blob_expansion']
    skey = lrb.sequences.keys()[snum]
    seq = lrb.sequences[skey]
    eta_L = np.rad2deg(np.array(seq.seq_record['wb_mtrx']['eta_L']))
    theta_L = np.rad2deg(np.array(seq.seq_record['wb_mtrx']['theta_L']))
    phi_L = np.rad2deg(np.array(seq.seq_record['wb_mtrx']['phi_L']))
    ephys = np.array(seq.seq_record['wb_mtrx']['AMsysCh1'])
    numstrks = shape(eta_L)[0]
    my_cmap = matplotlib.colors.LinearSegmentedColormap('my_colormap',cdict,256)
    cbar = arange(numstrks)[:,newaxis]
    phases = np.linspace(-2*pi,4*pi,1500)
    for strk in range(numstrks)[:46]:
        c = my_cmap((strk/(float(numstrks))))
        ax0 = subplot(2,1,1)
        plot(phi_L[strk,500:1000],theta_L[strk,500:1000],color = c,alpha = alpha)
        gca().set_ybound(-50,50)
        gca().set_yticks(linspace(-100,100,5))
        gca().set_xbound(-50,50)
        gca().set_xticks(linspace(-100,100,5))
        subplot(2,1,2)
        plot(phases,ephys[strk,:],color = c,alpha = alpha)
        gca().set_ybound(-100,100)
        gca().set_yticks(linspace(-100,100,5))
    gca().set_xticks([-2*pi,-1*pi,0,pi,2*pi,3*pi,4*pi])
    gca().set_xbound([-2*pi,4*pi])

In [53]:
my_cmap(0.5)

ValueError: data mapping points must start with x=0. and end with x=1

In [5]:
fly_db.close()

<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>


In [198]:
b1_spikes>phys_times[0]

array([False, False, False, ...,  True,  True,  True], dtype=bool)

KeyError: "unable to open object (Symbol table: Can't open object)"

In [60]:
#block to plot the data fit to the forier time series
ipsi_group = list()
contra_group = list()
for fly in swarm.flies[:10]:
    lrb = fly.experiments['lr_blob_expansion']
    side = lrb.exp_record['AMsysCh1_side'].value
    for skey in lrb.sequences.keys():
        seq = lrb.sequences[skey]
        if seq.seq_record['expan_pol'].value == 270.0:
            if side == 'r':
                ipsi_group.append(seq.seq_record['wb_mtrx'])
            if side == 'l':
                contra_group.append(seq.seq_record['wb_mtrx'])
        if seq.seq_record['expan_pol'].value == 90:
                contra_group.append(seq.seq_record['wb_mtrx'])
            if side = 'l':
                ipsi_group.append(seq.seq_record['wb_mtrx']
ax0 = subplot(3,1,1)
#############################
stroke_mtrx_r = [list() for x in range(45)]
stroke_mtrx_l = [list() for x in range(45)]
for nseq in ipsi_group:
    for i,stroke in enumerate(nseq['phi_L'][:45]):
        stroke_mtrx_r[i].append(rad2deg(stroke[500:1000]))
    for i,stroke in enumerate(nseq['phi_R'][:45]):
        stroke_mtrx_l[i].append(rad2deg(stroke[500:1000]))
for i,stroke_stack in enumerate(stroke_mtrx_r):
    phases = linspace(0,2.0*pi,500)+2*pi*i
    stroke_stack = rad2deg(stroke_stack)
    p = flylib.fit_fourier(stroke_stack,p_amp)
    plot(phases,flylib.fourier(phases,p),color = 'r')
for i,stroke_stack in enumerate(stroke_mtrx_l):
    phases = linspace(0,2.1*pi,500)+2*pi*i
    stroke_stack = rad2deg(stroke_stack)
    p = flylib.fit_fourier(stroke_stack,p_amp)
    plot(phases,flylib.fourier(phases,p),color = 'b')
############################
subplot(3,1,2,sharex = ax0)
stroke_mtrx_r = [list() for x in range(45)]
stroke_mtrx_l = [list() for x in range(45)]
for nseq in ipsi_group:
    for i,stroke in enumerate(nseq['theta_L'][:45]):
        stroke_mtrx_r[i].append(rad2deg(stroke[500:1000]))
    for i,stroke in enumerate(nseq['theta_R'][:45]):
        stroke_mtrx_l[i].append(rad2deg(stroke[500:1000]))
for i,stroke_stack in enumerate(stroke_mtrx_r):
    phases = linspace(0,2.0*pi,500)+2*pi*i
    stroke_stack = rad2deg(stroke_stack)
    p = flylib.fit_fourier(stroke_stack,p_dev)
    plot(phases,flylib.fourier(phases,p),color = 'r')
for i,stroke_stack in enumerate(stroke_mtrx_l):
    phases = linspace(0,2.0*pi,500)+2*pi*i
    stroke_stack = rad2deg(stroke_stack)
    p = flylib.fit_fourier(stroke_stack,p_dev)
    plot(phases,flylib.fourier(phases,p),color = 'b')
#############################
subplot(3,1,3,sharex = ax0)
stroke_mtrx_r = [list() for x in range(45)]
stroke_mtrx_l = [list() for x in range(45)]
for nseq in ipsi_group:
    for i,stroke in enumerate(nseq['eta_L'][:45]):
        stroke_mtrx_r[i].append(rad2deg(stroke[500:1000]))
    for i,stroke in enumerate(nseq['eta_R'][:45]):
        stroke_mtrx_l[i].append(rad2deg(stroke[500:1000]))
for i,stroke_stack in enumerate(stroke_mtrx_r):
    phases = linspace(0,2.0*pi,500)+2*pi*i
    stroke_stack = rad2deg(stroke_stack)
    p = flylib.fit_fourier(stroke_stack,p_rot)
    plot(phases,flylib.fourier(phases,p),color = 'r')
for i,stroke_stack in enumerate(stroke_mtrx_l):
    phases = linspace(0,2.0*pi,500)+2*pi*i
    stroke_stack = rad2deg(stroke_stack)
    p = flylib.fit_fourier(stroke_stack,p_rot)
    plot(phases,flylib.fourier(phases,p),color = 'b')

IndentationError: unindent does not match any outer indentation level (<ipython-input-60-23c11f6a5fff>, line 16)

In [152]:
p_str = [3.9,
         65.0445,4.2642,
         3.5806,-2.9492,
         0.1319,0.3639,
         0.7844,0.2098]
p_rot = [4.2936,
     1.8536,59.6529,
     5.1852,1.6095,
     -8.4569,9.8057,
     3.9562,5.8064,
     -3.0337,-2.8749,
     -2.8771,0.6686,
     0.8649,-0.6137,
     0.0771,-1.0007]

p_dev = [-6.173,
         6.5682,   3.7788,
         6.1335,   0.8717,
         -0.352,   -0.6606,
         0.0447,   1.2816,
         0.5257,   0.3556,
         -0.3202,  -0.2039,
         -0.2993,  0.1891,
         0.0221,   0.0379]

In [16]:
lrb.exp_record['AMsysCh1_side'].value

'r'

In [61]:
#block to plot the data fit to the forier time series
right_ipsi_group = list()
left_ipsi_group = list()

for fly in swarm.flies[:4]:
    lrb = fly.experiments['lr_blob_expansion']
    side = lrb.exp_record['AMsysCh1_side'].value
    if side == 'r':
        for skey in lrb.sequences.keys():
            seq = lrb.sequences[skey]
            if seq.seq_record['expan_pol'].value == 270.0:
                right_ipsi_group.append(seq.seq_record['wb_mtrx'])
    if side == 'l':
        for skey in lrb.sequences.keys():
            seq = lrb.sequences[skey]
            if seq.seq_record['expan_pol'].value == 90.0:
                left_ipsi_group.append(seq.seq_record['wb_mtrx'])
            
ax0 = subplot(3,1,1)
#############################
stroke_mtrx = [list() for x in range(45)]
for nseq in right_ipsi_group:
    for i,stroke in enumerate(nseq['phi_R'][:45]):
        stroke_mtrx[i].append(rad2deg(stroke[0:1500]))

for nseq in left_ipsi_group:
    for i,stroke in enumerate(nseq['phi_L'][:45]):
        stroke_mtrx[i].append(rad2deg(stroke[0:1500]))
    
    
#for i,stroke_stack in enumerate(stroke_mtrx):
#    phases = linspace(0,4.0*pi,500)
#    stroke_stack = rad2deg(stroke_stack)
#    p = flylib.fit_fourier(stroke_stack,p_amp)
#    plot(phases,flylib.fourier(phases,p),color = cm.jet_r(i/45.0))
"""
############################
subplot(3,1,2,sharex = ax0)
stroke_mtrx_r = [list() for x in range(45)]
stroke_mtrx_l = [list() for x in range(45)]
for nseq in ipsi_group:
    for i,stroke in enumerate(nseq['theta_L'][:45]):
        stroke_mtrx_r[i].append(rad2deg(stroke[500:1000]))
    for i,stroke in enumerate(nseq['theta_R'][:45]):
        stroke_mtrx_l[i].append(rad2deg(stroke[500:1000]))
for i,stroke_stack in enumerate(stroke_mtrx_r):
    phases = linspace(0,2.0*pi,500)+2*pi*i
    stroke_stack = rad2deg(stroke_stack)
    p = flylib.fit_fourier(stroke_stack,p_dev)
    plot(phases,flylib.fourier(phases,p),color = 'r')
for i,stroke_stack in enumerate(stroke_mtrx_l):
    phases = linspace(0,2.0*pi,500)+2*pi*i
    stroke_stack = rad2deg(stroke_stack)
    p = flylib.fit_fourier(stroke_stack,p_dev)
    plot(phases,flylib.fourier(phases,p),color = 'b')
#############################
subplot(3,1,3,sharex = ax0)
stroke_mtrx_r = [list() for x in range(45)]
stroke_mtrx_l = [list() for x in range(45)]
for nseq in ipsi_group:
    for i,stroke in enumerate(nseq['eta_L'][:45]):
        stroke_mtrx_r[i].append(rad2deg(stroke[500:1000]))
    for i,stroke in enumerate(nseq['eta_R'][:45]):
        stroke_mtrx_l[i].append(rad2deg(stroke[500:1000]))
for i,stroke_stack in enumerate(stroke_mtrx_r):
    phases = linspace(0,2.0*pi,500)+2*pi*i
    stroke_stack = rad2deg(stroke_stack)
    p = flylib.fit_fourier(stroke_stack,p_rot)
    plot(phases,flylib.fourier(phases,p),color = 'r')
for i,stroke_stack in enumerate(stroke_mtrx_l):
    phases = linspace(0,2.0*pi,500)+2*pi*i
    stroke_stack = rad2deg(stroke_stack)
    p = flylib.fit_fourier(stroke_stack,p_rot)
    plot(phases,flylib.fourier(phases,p),color = 'b')
"""

"\n############################\nsubplot(3,1,2,sharex = ax0)\nstroke_mtrx_r = [list() for x in range(45)]\nstroke_mtrx_l = [list() for x in range(45)]\nfor nseq in ipsi_group:\n    for i,stroke in enumerate(nseq['theta_L'][:45]):\n        stroke_mtrx_r[i].append(rad2deg(stroke[500:1000]))\n    for i,stroke in enumerate(nseq['theta_R'][:45]):\n        stroke_mtrx_l[i].append(rad2deg(stroke[500:1000]))\nfor i,stroke_stack in enumerate(stroke_mtrx_r):\n    phases = linspace(0,2.0*pi,500)+2*pi*i\n    stroke_stack = rad2deg(stroke_stack)\n    p = flylib.fit_fourier(stroke_stack,p_dev)\n    plot(phases,flylib.fourier(phases,p),color = 'r')\nfor i,stroke_stack in enumerate(stroke_mtrx_l):\n    phases = linspace(0,2.0*pi,500)+2*pi*i\n    stroke_stack = rad2deg(stroke_stack)\n    p = flylib.fit_fourier(stroke_stack,p_dev)\n    plot(phases,flylib.fourier(phases,p),color = 'b')\n#############################\nsubplot(3,1,3,sharex = ax0)\nstroke_mtrx_r = [list() for x in range(45)]\nstroke_mtrx_l 

In [38]:
for i,mtrx in enumerate(stroke_mtrx):
    plot(np.mean(np.array(mtrx),axis = 0),color = cm.jet(i/45.))

In [72]:
seq.seq_record['wb_mtrx'].keys()

[u'AMsysCh1',
 u'CamSync',
 u'LeftWing',
 u'Ph0',
 u'Ph1',
 u'Ph2',
 u'Photostim',
 u'RightWing',
 u'WBSync',
 u'Xpos',
 u'Ypos',
 u'axon_sample_times',
 u'eta_L',
 u'eta_R',
 u'phi_L',
 u'phi_R',
 u'photron_sample_times',
 u'stim_frame_times',
 u'theta_L',
 u'theta_R']

In [5]:
ephys_group = [111,112,114,115,116,117,118,122,123,124,125]
swarm = flylib.Squadron(fly_db,ephys_group)
global_wbmtrx = list()
for fly in swarm.flies:
    lrb = fly.experiments['lr_blob_expansion']
    for seq_key in lrb.sequences.keys():
        seq = lrb.sequences[seq_key]
        global_wbmtrx.append(seq.seq_record['wb_mtrx'])

In [110]:
exp = swarm.flies[5].experiments['lr_blob_expansion']

In [128]:
stroke

u'AMsysCh1'

In [124]:
ipsi_group[0].keys()

[u'AMsysCh1',
 u'CamSync',
 u'LeftWing',
 u'Ph0',
 u'Ph1',
 u'Ph2',
 u'Photostim',
 u'RightWing',
 u'WBSync',
 u'Xpos',
 u'Ypos',
 u'axon_sample_times',
 u'eta_L',
 u'eta_R',
 u'phi_L',
 u'phi_R',
 u'photron_sample_times',
 u'stim_frame_times',
 u'theta_L',
 u'theta_R']

In [52]:
stroke_bins = np.linspace(-0.2,0,50)

In [75]:
stimes = [np.array(mtrx['stim_frame_times']) for mtrx in global_wbmtrx]

In [78]:
mean_times = nanmean(vstack(stimes),axis = 1)

In [82]:
hist(mean_times,bins = 50)

(array([ 36.,  64.,  65.,  64.,  65.,  67.,  61.,  67.,  63.,  66.,  64.,
        64.,  69.,  66.,  64.,  62.,  67.,  62.,  66.,  61.,  68.,  67.,
        62.,  64.,  62.,  66.,  66.,  68.,  64.,  66.,  62.,  66.,  64.,
        66.,  64.,  64.,  67.,  67.,  64.,  63.,  66.,  65.,  64.,  64.,
        64.,  53.,  13.,   3.,   1.,   2.]),
 array([-0.19885315, -0.19456805, -0.19028294, -0.18599784, -0.18171273,
       -0.17742763, -0.17314252, -0.16885742, -0.16457231, -0.16028721,
       -0.1560021 , -0.151717  , -0.14743189, -0.14314679, -0.13886168,
       -0.13457658, -0.13029147, -0.12600637, -0.12172126, -0.11743616,
       -0.11315105, -0.10886595, -0.10458084, -0.10029574, -0.09601063,
       -0.09172553, -0.08744042, -0.08315532, -0.07887021, -0.07458511,
       -0.0703    , -0.0660149 , -0.06172979, -0.05744469, -0.05315958,
       -0.04887448, -0.04458937, -0.04030427, -0.03601916, -0.03173406,
       -0.02744895, -0.02316385, -0.01887874, -0.01459364, -0.01030853,
       -0.006